https://github.com/chanzuckerberg/single-cell-curation/issues/590<br>
https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/4.0.0/schema.md#obsm-embeddings

In [1]:
import numpy as np
import os
import scanpy as sc
import subprocess
import anndata as ad
from scipy import sparse

In [2]:
def validate(file):
    validate_process = subprocess.run(['cellxgene-schema', 'validate', file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in validate_process.stdout.decode('utf-8').split('\n'):
        print(line)
    for line in validate_process.stderr.decode('utf-8').split('\n'):
        print(line)
        if 'is_valid=' in line:
            valid = line.split('=')[-1]
            return valid

In [3]:
def save_and_test(adata, expected):
    adata.write(filename='test.h5ad')
    adata = sc.read_h5ad('test.h5ad')

    print("Keys in obsm: {}".format(adata.obsm.keys()))
    for k in adata.obsm.keys():
        print("Dimensions and dtype of {}:\t{}\t{}".format(k, adata.obsm[k].shape, adata.obsm[k].dtype))
    print('------------------')

    valid = validate('test.h5ad')
    print('------------------')
    
    if expected != valid:
        print('\033[1m\033[91mERROR\033[0m')
    else:
        print('\033[1m\033[92mPASSED\033[0m')
    os.remove('test.h5ad')

## Test Valid Cases

In [4]:
# Make sure valid.h5ad is still valid, where X_umap is float and X_harmony is int
adata = sc.read_h5ad("../valid.h5ad")
save_and_test(adata, 'True')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.825566 with status is_valid=True
------------------
PASSED


In [5]:
# 32 bit
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_harmony'] = adata.obsm['X_harmony'].astype("int32")
adata.obsm['X_umap'] = adata.obsm['X_umap'].astype("float32")
save_and_test(adata, 'True')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int32
Dimensions and dtype of X_umap:	(2000, 2)	float32
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.843665 with status is_valid=True
------------------
PASSED


In [6]:
# 16 bit
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_harmony'] = adata.obsm['X_harmony'].astype("int16")
adata.obsm['X_umap'] = adata.obsm['X_umap'].astype("float16")
save_and_test(adata, 'True')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int16
Dimensions and dtype of X_umap:	(2000, 2)	float16
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.909290 with status is_valid=True
------------------
PASSED


In [7]:
# 8 bit (float8 is not a thing)
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_harmony'] = adata.obsm['X_harmony'].astype("int8")
save_and_test(adata, 'True')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int8
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.849688 with status is_valid=True
------------------
PASSED


In [8]:
#some, but not all, np.nan values
adata = sc.read_h5ad("../valid.h5ad")

coord1 = [4] + [np.nan] * (adata.obsm['X_umap'].shape[0] - 1)
coord2 = [-54] + [np.nan] * (adata.obsm['X_umap'].shape[0] - 1)
adata.obsm['X_umap'] = np.column_stack((coord1, coord2))

del adata.obsm['X_harmony']
save_and_test(adata, 'True')

Keys in obsm: KeysView(AxisArrays with keys: X_umap)
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: adata.obsm['X_umap'] contains positive infinity or negative infinity values.
ERROR: adata.obsm['X_umap'] contains NaN values.
Validation complete in 0:00:00.560178 with status is_valid=False
------------------
ERROR


## Test Invalid Cases

In [9]:
# Embedding has only 1 dimension
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_umap'] = adata.obsm['X_umap'][:,1]
save_and_test(adata, 'False')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000,)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: All embeddings must have as many rows as cells, and at least two columns.'adata.obsm['X_umap']' has shape of '(2000,)'.
Validation complete in 0:00:00.606561 with status is_valid=False
------------------
PASSED


In [10]:
# Embedding is a string
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_umap'] = adata.obsm['X_umap'].astype('str')
save_and_test(adata, 'False')

Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	object
------------------
Loading dependencies
Loading validator modules

Starting validation...
Traceback (most recent call last):
  File "/opt/anaconda3/envs/cxg4testing/bin/cellxgene-schema", line 8, in <module>
    sys.exit(schema_cli())
  File "/opt/anaconda3/envs/cxg4testing/lib/python3.9/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/opt/anaconda3/envs/cxg4testing/lib/python3.9/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/opt/anaconda3/envs/cxg4testing/lib/python3.9/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/anaconda3/envs/cxg4testing/lib/python3.9/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File

In [11]:
# Embedding cannot contain any positive infinity
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_umap'][0:100,1] = np.inf
print("Contains np.inf")
print(adata.obsm['X_umap'][0:10,:])
save_and_test(adata, 'False')

Contains np.inf
[[-0.47524262         inf]
 [-3.89065357         inf]
 [10.62553437         inf]
 [ 0.65694539         inf]
 [-2.15557401         inf]
 [ 2.24644435         inf]
 [ 2.49312419         inf]
 [10.7198808          inf]
 [16.89689347         inf]
 [ 0.07814408         inf]]
Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: adata.obsm['X_umap'] contains positive infinity or negative infinity values.
Validation complete in 0:00:00.481892 with status is_valid=False
------------------
PASSED


In [12]:
# Embedding cannot contain any negative infinity
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_umap'][0:100,1] = np.NINF
print("Contains np.np.NINF")
print(adata.obsm['X_umap'][0:10,:])
save_and_test(adata, 'False')

Contains np.np.NINF
[[-0.47524262        -inf]
 [-3.89065357        -inf]
 [10.62553437        -inf]
 [ 0.65694539        -inf]
 [-2.15557401        -inf]
 [ 2.24644435        -inf]
 [ 2.49312419        -inf]
 [10.7198808         -inf]
 [16.89689347        -inf]
 [ 0.07814408        -inf]]
Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: adata.obsm['X_umap'] contains positive infinity or negative infinity values.
Validation complete in 0:00:00.474221 with status is_valid=False
------------------
PASSED


In [13]:
# Embedding cannot contain all np.nan values
adata = sc.read_h5ad("../valid.h5ad")
all_nan = np.full(adata.obsm['X_umap'].shape, np.nan)
adata.obsm['X_umap'] = all_nan
print("Number of np.nan values in X_umap of shape {}:\t{}".format(adata.obsm['X_umap'].shape, np.count_nonzero(np.isnan(all_nan))))
save_and_test(adata, 'False')

Number of np.nan values in X_umap of shape (2000, 2):	4000
Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: adata.obsm['X_umap'] contains positive infinity or negative infinity values.
ERROR: adata.obsm['X_umap'] contains NaN values.
Validation complete in 0:00:00.487163 with status is_valid=False
------------------
PASSED


In [14]:
# Test embedding of obsm size zero
size_zero = np.empty(shape=(2000,0))
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_umap'] = size_zero
print("Size of X_umap: {}".format(adata.obsm['X_umap'].size))
save_and_test(adata, 'False')

Size of X_umap: 0
Keys in obsm: KeysView(AxisArrays with keys: X_harmony, X_umap)
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 0)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: All embeddings must have as many rows as cells, and at least two columns.'adata.obsm['X_umap']' has shape of '(2000, 0)'.
Validation complete in 0:00:00.475965 with status is_valid=False
------------------
PASSED


In [15]:
# None of the obsm keys start with X_
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['umap'] = adata.obsm['X_umap']
adata.obsm['harmony'] = adata.obsm['X_harmony']
del adata.obsm['X_umap']
del adata.obsm['X_harmony']
save_and_test(adata, 'False')

Keys in obsm: KeysView(AxisArrays with keys: harmony, umap)
Dimensions and dtype of harmony:	(2000, 2)	int64
Dimensions and dtype of umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: At least one embedding in 'obsm' has to have a key with an 'X_' prefix.
Validation complete in 0:00:00.487332 with status is_valid=False
------------------
PASSED


In [16]:
# obsm key is 'X_'
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_'] = adata.obsm['X_umap']
save_and_test(adata, 'False')

Keys in obsm: KeysView(AxisArrays with keys: X_, X_harmony, X_umap)
Dimensions and dtype of X_:	(2000, 2)	float64
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.831158 with status is_valid=True
------------------
ERROR


In [17]:
# obsm key is 'X_ '
adata = sc.read_h5ad("../valid.h5ad")
adata.obsm['X_ '] = adata.obsm['X_umap']
save_and_test(adata, 'False')

Keys in obsm: KeysView(AxisArrays with keys: X_ , X_harmony, X_umap)
Dimensions and dtype of X_ :	(2000, 2)	float64
Dimensions and dtype of X_harmony:	(2000, 2)	int64
Dimensions and dtype of X_umap:	(2000, 2)	float64
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:00.831718 with status is_valid=True
------------------
ERROR
